In [5]:

# import packages for analysis and modeling
import pandas as pd  # data frame operations
import numpy as np  # arrays and math functions
from scipy.stats import uniform  # for training-and-test split
import statsmodels.api as sm  # statistical models (including regression)
import statsmodels.formula.api as smf  # R-like model specification
import matplotlib.pyplot as plt  # 2D plotting
import re # regular expression for string operations
import seaborn as sns  # PROVIDES TRELLIS AND SMALL MULTIPLE PLOTTING

# read in coaches8.csv and  create data frame
coaches = pd.read_csv("coaches.csv")

In [6]:
#coaches.iloc[:,:9].to_csv('coaches.csv')
coaches.head()

,Unnamed: 0,School,Conference,Coach,SchoolPay,TotalPay,Bonus,BonusPaid,AssistantPay,Buyout
0,0,Air Force,Mt. West,Troy Calhoun,885000,885000,247000,0,0,0
1,1,Akron,MAC,Terry Bowden,411000,412500,225000,50000,0,688500
2,2,Alabama,SEC,Nick Saban,8307000,8307000,1100000,500000,0,33600000
3,3,Alabama at Birmingham,C0USA,Bill Clark,900000,900000,950000,165471,0,3847500
4,4,Appalachian State,Sun Belt,Scott Satterfield,712500,712500,295000,145000,0,2160417


In [7]:
# Get additional data stadium capacity from relevant websites using webscrabbing
# Inherit and Modify HTMLParser class to get the relevant data from websites related to capacity
from html.parser import HTMLParser

class HTMLParser2(HTMLParser):
    import re
    
    def __init__(self):
        HTMLParser.__init__(self)
        self.counter=0
        self.capacity_min_limit=10000
        self.capacity_max_limit=200000
        self.struct_raw=[]
        self.struct_num=[]
        
    def feed(self,data):
        self.struct_raw=[]
        self.struct_num=[]
        self.counter=0
        HTMLParser.feed(self,data)
        self.struct_raw.append('{} lines contains count out of range ({},{})'.format(self.counter,self.capacity_min_limit,self.capacity_max_limit))
        #print(self.struct)
        return self.struct_raw,self.struct_num
        
    def handle_starttag(self, tag, attrs):
        if re.search('capacity',tag):
            print("Start tag: ", tag)
            for attr in attrs:
                print("attr:", attr)
    def handle_endtag(self, tag):
        if re.search('capacity',tag):
            print("End tag: ", tag)
    def handle_comment(self, data):
        if re.search('capacity',data):
            print("Comment: ", data)
    def handle_data(self, data):
        if re.search('capacity',data):
            try:
                m = re.search('(capacity)(\D+)(\d+)+', data.replace(',',''))
                if ((int(m.group(3))> self.capacity_min_limit) & (int(m.group(3))< self.capacity_max_limit)) :
                    self.struct_raw.append((m.group()))
                    self.struct_num.append(int(m.group(3)))
                    #print("Data: ", m.group())
                else:
                    self.counter+=1
            except:
                pass


    #def print_result(self):
        #print(self.struct,'\n','{} lines contains count out of range (10000,200000)'.format(self.counter))

parser = HTMLParser2()
x,y=parser.feed("<html><head><title>Coder</title></head><body><h1><!--hi-->Opening in 1962, the stadium cost $3.5 million. Named after the team, Falcon Stadium has a seating capacity of just over 52,000. On September 22, 1962 the Falcons beat Colorado Springs in the inaugural first game at the stadium.capacity of over 100,000 The lower grandstand encloses the playing field while the second and third decks are located on one side of the stadium.  The main scoreboard sits above the stands in the endzone. Falcon Stadium has changed very little over the years, but has hosted many memorable football games. Lighting was added in 2002 and a new videoboard in 2004.</h1></body></html>")
print(x,y)


['capacity of just over 52000', '0 lines contains count out of range (10000,200000)'] [52000]


In [8]:
# using google search to scan for relevant information for each shool stadium . select 10 results for each school and iterate 
# through the webpage untill we get the capacity information with the help of above HTMLParser class

from googlesearch import search
import urllib
from bs4 import BeautifulSoup
import re

#search(query, tld='com', lang='en', num=10, start=0, stop=None, pause=2.0)
# to search 
query_lst=[x+' school stadium capacity' for x in coaches['School']]

for index,query in enumerate(query_lst):
    #if index<10:
        for url in search(query, tld="com", num=1, stop=1, pause=2): 
            coaches.loc[index,'url']=url
            try:
                with urllib.request.urlopen(url) as request:
                    stadiums=request.read().decode('utf-8').replace('\0', '')
                    #print(stadiums)
                    raw_data=parser.feed(stadiums)
                    #print(type(raw_data))
                    if max(raw_data[1])>0:
                        coaches.at[index,'capacity_raw_data']=raw_data[0]
                        coaches.at[index,'capacity']=max(raw_data[1])
                        print(url,raw_data[0],raw_data[1])
                        break
            except:
                try:
                    coaches.at[index,'capacity_raw_data']=('HTTPError on '+ url)
                except:
                    pass

https://en.wikipedia.org/wiki/Bryant%E2%80%93Denny_Stadium ['capacity of 12000', 'capacity to 24000', 'capacity to 31000', 'capacity to approximately 101600', 'capacity of 101821', '1 lines contains count out of range (10000,200000)'] [12000, 24000, 31000, 101600, 101821]
https://en.wikipedia.org/wiki/Legion_Field ['capacity to 54600', '2 lines contains count out of range (10000,200000)'] [54600]
https://en.wikipedia.org/wiki/Kidd_Brewer_Stadium ['capacity was expanded to 18000', '1 lines contains count out of range (10000,200000)'] [18000]
https://en.wikipedia.org/wiki/Arizona_Stadium ['capacity of 50782', '1 lines contains count out of range (10000,200000)'] [50782]
https://en.wikipedia.org/wiki/Sun_Devil_Stadium ['capacity was 30000', 'capacity to the 55000', 'capacity to 65870', '1 lines contains count out of range (10000,200000)'] [30000, 55000, 65870]
https://en.wikipedia.org/wiki/War_Memorial_Stadium_(Arkansas) ['capacity of 31075', '0 lines contains count out of range (10000,20

https://en.wikipedia.org/wiki/Maryland_Stadium ['capacity of 34680', 'capacity to 65000', 'capacity from 51500', '0 lines contains count out of range (10000,200000)'] [34680, 65000, 51500]
https://en.wikipedia.org/wiki/Warren_McGuirk_Alumni_Stadium ["capacity is only modestly above the NCAA's requirement of 15000", '0 lines contains count out of range (10000,200000)'] [15000]
https://en.wikipedia.org/wiki/Crump_Stadium ['capacity of 25000', 'capacity to 15000', 'capacity to over 45000', '2 lines contains count out of range (10000,200000)'] [25000, 15000, 45000]
https://en.wikipedia.org/wiki/Yager_Stadium_(Miami_University) ['capacity of 24286', '0 lines contains count out of range (10000,200000)'] [24286]
https://en.wikipedia.org/wiki/Michigan_Stadium ['capacity is 107601', 'capacity of 72000', 'capacity to be expanded beyond 100000', 'capacity of 72000', 'capacity to 101001', 'capacity of 109901', 'capacity was 72000', 'capacity of over 150000', '2 lines contains count out of range (1

https://en.wikipedia.org/wiki/Bobcat_Stadium_(Texas_State) ['capacity grew to over 16000', 'capacity to 30000', '0 lines contains count out of range (10000,200000)'] [16000, 30000]
https://en.wikipedia.org/wiki/Jones_AT%26T_Stadium ['capacity of 48000', 'capacity to 60454', 'capacity at that time to 58930', '0 lines contains count out of range (10000,200000)'] [48000, 60454, 58930]
https://en.wikipedia.org/wiki/Sun_Bowl_(stadium) ['capacity of 52000', 'capacity to its current figure of 51500', '0 lines contains count out of range (10000,200000)'] [52000, 51500]
https://en.wikipedia.org/wiki/Yulman_Stadium ['capacity of 30000', '0 lines contains count out of range (10000,200000)'] [30000]
https://en.wikipedia.org/wiki/LaVell_Edwards_Stadium ['capacity to 45000', 'capacity to 64045', '0 lines contains count out of range (10000,200000)'] [45000, 64045]
https://en.wikipedia.org/wiki/Maverik_Stadium ['capacity of the stadium to 30257', 'capacity was temporarily reduced to 22059', '0 lines c